# Beautiful Soup Tutorial

Como científico de datos, tarde o temprano llegarás a un punto en el que tendrás que recopilar grandes cantidades de datos. Ya sea un proyecto o por pasatiempo y no siempre podremos contar con las API, pero tranquilo tenemos el web scraping... ¡Y una de las mejores herramientas de web scraping es Beautiful Soup!

## ¿Pero.... qué es el web scraping?

En pocas palabras, el web scraping es la recopilación automatizada de datos de sitios web (para ser más precisos, del contenido HTML de los sitios web).

En este Jupyter, aprenderás los conceptos básicos sobre cómo extraer datos de HTML. 

Lo harás extrayendo datos de la página animeseries.io, y para lograr esto, también tendrá que hacer uso de un poco de pandas principalmente..

### Conoce a tus nuevos mejores amigos: 

- Beautiful Soup
- Requests

In [ ]:
# !pip install beautifulsoup4

Para obtener la experiencia completa de Beautiful Soup, también deberás instalar un parser, dentro de ellos tenemos..

- html.parser
- lxml
- html5lib


Vamos a utilizar el lxml ya que es el mas rápido 

In [ ]:
# !pip install lxml

Se necesita una cosa más para que podamos comenzar a hacer web scraping, y es la biblioteca de ```requests```. Con ```requests``` podemos solicitar páginas web de sitios web.

In [ ]:
# !pip install requests

Ahora asi manos a la obra..

## Mi primer scraping

Como siempre lo primero es importar las librerías 

In [ ]:
from bs4 import BeautifulSoup as bs
from string import ascii_uppercase
import requests
import pandas as pd
import numpy as np
import re

Ahora, estamos listos para solicitar nuestra primera página web. No es nada complicado: guardamos la URL que queremos raspar en la variable URL, luego solicitamos la URL (requests.get (url)) y guardamos la respuesta en la variable de respuesta:

In [ ]:
url = "https://animeseries.so/popular-anime"
response = requests.get(url)

Cómo saber si se guardo correctamente el sitio web?

In [ ]:
print(response.status_code)

Posibles respuestas:

- [Respuestas informativas](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status#information_responses) (100–199)
- [Respuestas exitosas](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status#successful_responses) (200–299)
- [Mensajes de redirección](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status#redirection_messages) (300–399)
- [Respuestas de error del cliente](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status#client_error_responses) (400–499)
- [Respuestas de error del servidor](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status#server_error_responses) (500–599)

Pero necesitamos el contenido HTML de la página web solicitada, así que como siguiente paso guardamos el contenido de la respuesta a html:

In [ ]:
html = response.content

Lo podemos imprimir para ver su estructura

In [ ]:
html

Este es el resultado obtenido en HTML de la página, pero es realmente difícil de leer...

Pero para eso usamos BeautifulSoup

Cómo lo hacemos?..

Creamos un objeto BeautifulSoup llamado soup con la siguiente línea de código:

In [ ]:
soup = bs(html, "html.parser")

bs?

> from bs4 import BeautifulSoup as bs

El primer parámetro del método bs() es html (que fue la variable en la que guardamos ese contenido HTML difícil de leer de la URL de los libros más vendidos)

El segundo parámetro ('html.parser'), es el parser que se usa en html 

Ahora vamos a ver el cambio

In [ ]:
print(soup)

## Cómo navegar por un objeto de Beautiful Soup

HTML consta de elementos como enlaces, párrafos, encabezados, bloques, etc. Estos elementos están envueltos entre etiquetas; dentro de la etiqueta de apertura y cierre se puede encontrar el contenido del elemento.

![image](img\html-content-web-scraping.png)

Los elementos HTML también pueden tener atributos que contienen información adicional sobre el elemento. Los atributos se definen en las etiquetas de apertura con la siguiente sintaxis: nombre del atributo = "valor del atributo".

![image](img\attribute-example-for-web-scraping-1536x386.png)

Ahora que hemos aprendido algo de HTML básico, finalmente podemos comenzar a extraer datos de soup. Simplemente escriba un nombre de etiqueta después de soup y un punto (como soup.title), y observe cómo se desarrolla la magia:

In [ ]:
soup.title

In [ ]:
soup.h1

In [ ]:
print(soup.h1)

Y sí queremos solo el texto?..

In [ ]:
soup.h1.get_text()

¿Qué sucede si solo necesita el atributo de un elemento? Tampoco hay problema:

In [ ]:
soup.a

In [ ]:
soup.img['src']

In [ ]:
soup.img.get('src')

También podemos..
> soup.a.get("href")

La sintaxis de soup.```cualquier_etiqueta``` devuelve solo el primer elemento con ese nombre de etiqueta. En lugar de soup.```cualquier_etiqueta```, también puedes usar el método .find() y obtendrás exactamente el mismo resultado:

In [ ]:
print("Sin utilizar .find()")
print(soup.h1)
print("Utilizando .find()")
print(soup.find("h1"))

A menudo, no solo necesitas uno, sino todos los elementos (por ejemplo, cada enlace en una página). Para eso es bueno el método .find_all():

In [ ]:
soup.find_all('a')

Si nos fijamos podemos ver que lo que nos devuelve es una lista..

Qué podemos hacer con una lista?..

In [ ]:
all_a = soup.find_all('a')
for a in all_a[2:5]:
    print(a.text)

Ahora vamos a tomar el grid principal para buscar los animes en toda la página

In [ ]:
grid = soup.find_all('div', 
            class_ = 'content_episode revent datagrild')

Cantidad de paginas

In [ ]:
href_valor = soup.find_all('li', class_='last')[0].find('a').get('href')

numero_pagina = re.search(r'page=(\d+)', href_valor)

if numero_pagina:
    numero_pagina = numero_pagina.group(1)
numero_pagina

Se itera sobre la lista de elementos contenidos en el grid

In [ ]:
for items in grid[:1]:
    animes = items.find_all('div', class_ = 'name')
    url_anime = items.find_all('a')
    
anime = animes[0].text
anime_link = url_anime[0].get('href')

Se obtiene el la lista de animes y urls pero no esta completa

In [ ]:
print('Titulo: ', anime)
print('--*--'*20)
print('Link parcial: ', anime_link)

Para construir la url completa se necesita la url base

In [ ]:
base = 'https://animeseries.so'

Uniendo la url base con la obtenida anteriormente se completa la url de cada anime

In [ ]:
new_url = base + anime_link
new_url

Es necesario repetir los pasos anteriores para la nueva url

Se crea una Sopa nueva

In [ ]:
response = requests.get(new_url)
soup2 = bs(response.content, "html.parser")

Estando dentro de esta nueva página se vuelve a inspeccionar

Se tomara el recuadro mas externo porque es el que contiene toda la información

In [ ]:
main_body = soup2.find_all('div', class_ = 'main_body')

Para la imagen necesitamos el recuadro de la izquierda

In [ ]:
recuadro_izq = main_body[0].find_all('div', class_ = 'left')

Para la información el de la derecha

In [ ]:
recuadro_der = main_body[0].find_all('div', class_ = 'right')

Obtenemos la url de la imagen primero

In [ ]:
imagen = recuadro_izq[0].find('img').get('src')

Ahora con el recuadro derecho obtenemos la informacion:

- Descripción
- Otros nombres
- Pais
- Status
- Fecha
- Genero
- Tipo
- Temporada
- Episodios

Etiqueta

In [ ]:
etiqueta = recuadro_der[0]

Descripción

In [ ]:
descripcion = etiqueta.find('p').text
print('Descripcion: ',descripcion)

Otros Titulos

In [ ]:
elementos_p = etiqueta.find_all('p', class_='des')

texto_deseado = elementos_p[0].get_text(separator=' ', strip=True)

span_text = elementos_p[0].find('span').get_text(strip=True)
otros_titulos = texto_deseado.replace(span_text, '', 1).strip()
otros_titulos

Pais

In [ ]:
elementos_p = etiqueta.find_all('p', class_='des')

texto_deseado = elementos_p[1].get_text(separator=' ', strip=True)

span_text = elementos_p[1].find('span').get_text(strip=True)
pais = texto_deseado.replace(span_text, '', 1).strip()
pais

Status, Fecha, Generos

In [ ]:
temporada = list()
genero = list()
for i in etiqueta.find_all('a')[2:]:
    temporada.append(i.text)

In [ ]:
status, fecha = temporada[:2]
genero = temporada[2:]
genero = ', '.join(genero)

print('Status: ',status)
print('Fecha: ', fecha)
print('Generos: ',genero)

Episodios, Tipo

In [ ]:
listaEpisodios = main_body[0].find_all('div',class_='list_episode')

try:
    texto = listaEpisodios[0].find('span',class_='name').text
    texto = re.search(r'Episode [0-9]+',texto)
    episodios = re.search('[0-9]+', texto[0])[0]
    if int(episodios) > 0 and int(episodios) < 2:
        tipo = 'Pelicula'
    else:
        tipo = 'Serie'
except:
    episodios = None
    tipo = None

Temporada

In [ ]:
def season_word_to_number(word):
    word_to_number = {
        'first': '1',
        'second': '2',
        'third': '3',
        'fourth': '4',
        'fifth': '5',
    }
    return word_to_number.get(word.lower())

In [ ]:
match = re.search(r"Season (\d+)", anime)
if match:
    temporada = match.group(1)
else:
    temporada = None

if temporada is None:
    match_descripcion = re.search(r"(first|second|third|fourth|fifth) season", descripcion, re.IGNORECASE)
    if match_descripcion:
        temporada_palabra = match_descripcion.group(1)
        temporada = season_word_to_number(temporada_palabra)

In [ ]:
anime_dicc = {
    'Titulo': anime,
    'Alternativos': otros_titulos,
    'Descripcion': descripcion,
    'Status': status,
    'Genre': genero,
    'Tipo': tipo,  
    'Episodes': episodios,
    'Temporada': temporada,
    'Fecha': fecha,
    'Pais': pais,
    'Imagen': imagen,
    'URL': new_url
}

In [ ]:
pd.DataFrame([anime_dicc])

Teniendo todos los elementos realizamos al automatización

In [ ]:
anime_dicc = {
    'Titulo': [],
    'Alternativos': [],
    'Descripcion': [],
    'Status': [],
    'Genre': [],
    'Tipo': [],  
    'Episodios': [],
    'Temporada': [],
    'Fecha': [],
    'Pais': [],
    'Imagen': [],
    'URL': []
}

base = 'https://animeseries.so'


def season_word_to_number(word):
    word_to_number = {
        'first': '1',
        'second': '2',
        'third': '3',
        'fourth': '4',
        'fifth': '5',
    }
    return word_to_number.get(word.lower())

url = f"https://animeseries.so/popular-anime"
response = requests.get(url)
soup = bs(response.content, "html.parser")

href_valor = soup.find_all('li', class_='last')[0].find('a').get('href')

numero_pagina = re.search(r'page=(\d+)', href_valor)

if numero_pagina:
    numero_pagina = numero_pagina.group(1)

# for page in range(1,int(numero_pagina)+1):
for page in range(1,3):
    
    print(f"\rProcessing page {page}/{len(range(1, 3))}..")
    
    url = f"https://animeseries.so/popular-anime?page={page}"
    response = requests.get(url)
    soup = bs(response.content, "html.parser")

    grid = soup.find_all('div', class_ = 'content_episode revent datagrild')

    animes = [anime.text for anime in grid[0].find_all('div', class_ = 'name')]
    enlaces_filtrados = [a for a in grid[0].find_all('a') if a.has_attr('title')]
    animes_url = [base+a.get('href') for a in enlaces_filtrados]

    for ind,anime_url in enumerate(animes_url):
        
        print(f"\rProcessing {ind+1}/{len(animes_url)}..")
        
        response = requests.get(anime_url)
        soup2 = bs(response.content, "html.parser")
        main_body = soup2.find_all('div', class_ = 'main_body')
        
        recuadro_izq = main_body[0].find_all('div', class_ = 'left')
        imagen = recuadro_izq[0].find('img').get('src')
        
        recuadro_der = main_body[0].find_all('div', class_ = 'right')
        etiqueta = recuadro_der[0]
        descripcion = etiqueta.find('p').text
        elementos_p = etiqueta.find_all('p', class_='des')
        resultados = [(p.get_text(separator=' ', strip=True).replace(p.find('span').get_text(strip=True), '', 1).strip()) for p in elementos_p]
        otros_titulos = resultados[0]
        pais = resultados[1]
        enlaces = etiqueta.find_all('a')[2:]
        temporada = [i.text for i in enlaces]
        status, fecha, *generos = temporada
        genero = ', '.join(generos)
        listaEpisodios = main_body[0].find_all('div',class_='list_episode')
        try:
            texto = listaEpisodios[0].find('span',class_='name').text
            texto = re.search(r'Episode [0-9]+',texto)
            episodios = re.search('[0-9]+', texto[0])[0]
            if int(episodios) > 0 and int(episodios) < 2:
                tipo = 'Pelicula'
            else:
                tipo = 'Serie'
        except:
            episodios = None
            tipo = None
        
        anime_dicc['Titulo'].append(animes[ind])
        anime_dicc['Alternativos'].append(otros_titulos)
        anime_dicc['Descripcion'].append(descripcion)
        anime_dicc['Status'].append(status)
        anime_dicc['Genre'].append(genero)
        anime_dicc['Tipo'].append(tipo)  
        anime_dicc['Episodios'].append(episodios)
        anime_dicc['Temporada'].append(temporada)
        anime_dicc['Fecha'].append(fecha)
        anime_dicc['Pais'].append(pais)
        anime_dicc['Imagen'].append(imagen)
        anime_dicc['URL'].append(anime_url)
    
df = pd.DataFrame(anime_dicc)
print('\rWeb Scraping Complete')

In [ ]:
anime_dicc = {
    'Titulo': [],
    'Alternativos': [],
    'Descripcion': [],
    'Status': [],
    'Genre': [],
    'Tipo': [],  
    'Episodios': [],
    'Temporada': [],
    'Fecha': [],
    'Pais': [],
    'Imagen': [],
    'URL': []
}

base = 'https://animeseries.so'


def season_word_to_number(word):
    word_to_number = {
        'first': '1',
        'second': '2',
        'third': '3',
        'fourth': '4',
        'fifth': '5',
    }
    return word_to_number.get(word.lower())

letras = list(ascii_uppercase)
letras.insert(0,"special")

# for alphabet in letras:
for page in [letras[0]]:
    
    print(f"\rProcessing page {page}/{len([letras[0]])}..")
    
    url = f"https://animeseries.so/search/character={page}"
    response = requests.get(url)
    soup = bs(response.content, "html.parser")

    grid = soup.find_all('div', class_ = 'content_episode revent datagrild')

    animes = [anime.text for anime in grid[0].find_all('div', class_ = 'name')]
    enlaces_filtrados = [a for a in grid[0].find_all('a') if a.has_attr('title')]
    animes_url = [base+a.get('href') for a in enlaces_filtrados]

    for ind,anime_url in enumerate(animes_url):
        
        print(f"\rProcessing {ind+1}/{len(animes_url)}..")
        
        response = requests.get(anime_url)
        soup2 = bs(response.content, "html.parser")
        main_body = soup2.find_all('div', class_ = 'main_body')
        
        recuadro_izq = main_body[0].find_all('div', class_ = 'left')
        imagen = recuadro_izq[0].find('img').get('src')
        
        recuadro_der = main_body[0].find_all('div', class_ = 'right')
        etiqueta = recuadro_der[0]
        descripcion = etiqueta.find('p').text
        elementos_p = etiqueta.find_all('p', class_='des')
        resultados = [(p.get_text(separator=' ', strip=True).replace(p.find('span').get_text(strip=True), '', 1).strip()) for p in elementos_p]
        otros_titulos = resultados[0]
        pais = resultados[1]
        enlaces = etiqueta.find_all('a')[2:]
        temporada = [i.text for i in enlaces]
        status, fecha, *generos = temporada
        genero = ', '.join(generos)
        listaEpisodios = main_body[0].find_all('div',class_='list_episode')
        try:
            texto = listaEpisodios[0].find('span',class_='name').text
            texto = re.search(r'Episode [0-9]+',texto)
            episodios = re.search('[0-9]+', texto[0])[0]
            if int(episodios) > 0 and int(episodios) < 2:
                tipo = 'Pelicula'
            else:
                tipo = 'Serie'
        except:
            episodios = None
            tipo = None
        
        anime_dicc['Titulo'].append(animes[ind])
        anime_dicc['Alternativos'].append(otros_titulos)
        anime_dicc['Descripcion'].append(descripcion)
        anime_dicc['Status'].append(status)
        anime_dicc['Genre'].append(genero)
        anime_dicc['Tipo'].append(tipo)  
        anime_dicc['Episodios'].append(episodios)
        anime_dicc['Temporada'].append(temporada)
        anime_dicc['Fecha'].append(fecha)
        anime_dicc['Pais'].append(pais)
        anime_dicc['Imagen'].append(imagen)
        anime_dicc['URL'].append(anime_url)
    
df = pd.DataFrame(anime_dicc)
print('\rWeb Scraping Complete')